In [46]:
## The treshold adjusting 

In [47]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import numpy as np
import scipy.stats as stats

class WealthAgent(Agent):
    def __init__(self, unique_id, model, disposable_income, dwelling, technology, bad_dwelling=False, inability=False, arrears=False):
        super().__init__(unique_id, model)
        self.disposable_income = disposable_income
        self.dwelling = dwelling
        self.technology = technology
        self.bad_dwelling = bad_dwelling
        self.inability = inability
        self.arrears = arrears

    @property
    def energy_cost(self):
        return self.model.energy_price * (self.dwelling + self.technology)

    def check_arrears(self):
        if self.disposable_income < self.energy_cost:
            self.arrears = True
        else:
            self.arrears = False

    def check_inability(self):
        if self.energy_cost > self.model.inability_threshold * self.disposable_income:
            self.inability = True
        else:
            self.inability = False

    def step(self):
        self.check_arrears()
        self.check_inability()


class WealthModel(Model):
    @staticmethod
    def calculate_gini(incomes):
        incomes = np.sort(incomes)
        n = len(incomes)
        index = np.arange(1, n + 1)
        return ((np.sum((2 * index - n  - 1) * incomes)) / (n * np.sum(incomes)))

    @staticmethod
    def generate_income_distribution(num_people, median_income, gini_target):
        alpha = (gini_target + 1) / (2 - gini_target)
        for _ in range(10000):
            incomes = stats.gamma.rvs(alpha, scale=median_income/alpha, size=num_people)
            gini_current = WealthModel.calculate_gini(incomes)
            if np.isclose(gini_current, gini_target, atol=0.01):
                return incomes
            elif gini_current < gini_target:
                alpha *= 0.9
            else:
                alpha *= 1.1
        raise Exception(f"Failed to reach target Gini coefficient in 1000 iterations. Current Gini: {gini_current}")

    def __init__(self, N, median_income, gini_target, rate, energy_price_x, share_fuel_x, energy_price_y, share_fuel_y, inability_target):
        self.num_agents = N
        self.median_income = median_income
        self.gini_target = gini_target
        self.rate = rate
        self.energy_price_x = energy_price_x
        self.share_fuel_x = share_fuel_x
        self.energy_price_y = energy_price_y
        self.share_fuel_y = share_fuel_y
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            agent_reporters={"Dwelling": "dwelling", 
                             "Technology": "technology", 
                             "Income": "disposable_income", 
                             "Arrears": "arrears", 
                             "Inability": "inability",
                             "EnergyCost": "energy_cost"})
        self.inability_target = inability_target
        self.inability_threshold = 0.1  # Initialize inability_threshold to 0.1

        incomes = self.generate_income_distribution(self.num_agents, self.median_income, self.gini_target)
        
        for i in range(self.num_agents):
            a = WealthAgent(i, self, incomes[i], np.random.randint(500, 1800), np.random.randint(200, 600))
            self.schedule.add(a)

    @property
    def energy_price(self):
        return self.energy_price_x * self.share_fuel_x + self.energy_price_y * self.share_fuel_y
    def adjust_inability_threshold(self):
        inability_share = sum(agent.inability for agent in self.schedule.agents) / self.num_agents
        if inability_share < self.inability_target and self.inability_threshold < 1:
            self.inability_threshold += 0.001  # Decreased step size
        elif inability_share > self.inability_target and self.inability_threshold > 0:
            self.inability_threshold -= 0.001  # Decreased step size

    def step(self):
        self.schedule.step()
        self.adjust_inability_threshold()
        self.datacollector.collect(self)


model = WealthModel(100, 60000, 0.3, [0.01, 0.05], 1, 0.5, 1, 0.5, 0.2)  # added the inability_target parameter

for i in range(100):
    model.step()

agent_data = model.datacollector.get_agent_vars_dataframe()


In [48]:
model = WealthModel(100, 3000, 0.5, [0.01, 0.05], 1, 0.5, 1, 0.5,0.1)

for i in range(100):   # this is the simulation loop
    model.step()

agent_data = model.datacollector.get_agent_vars_dataframe()

In [49]:
agent_data.Inability.tail(99).sum()

99